<a href="https://colab.research.google.com/github/KarolZajac/ZTP_Proj/blob/main/Etap1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 

In [ ]:
from sklearn.linear_model import LogisticRegression

import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [99]:
pip install dask-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
pip install --upgrade dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2022.12.1
    Uninstalling dask-2022.12.1:
      Successfully uninstalled dask-2022.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.12.1 requires dask==2022.12.1, but you have dask 2023.5.0 which is incompatible.


In [102]:
pip install --upgrade dask-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO: pip is looking at multiple versions of distributed to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.6/966.6 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: distributed
    Found existing installation: distributed 2022.12.1
    Uninstalling distributed-2022.12.1:
      Successfully uninstalled distributed-2022.12.1


In [37]:
import dask.dataframe as dd 
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from dask_ml.preprocessing import LabelEncoder

data = dd.read_csv("bank.csv", header=0, delimiter=';')

data = data.dropna()
data["y"] = data["y"].replace({"yes": 1, "no": 0})

le = LabelEncoder()
data['job']=le.fit_transform(data['job'])
data['marital']=le.fit_transform(data['marital'])
data['education']=le.fit_transform(data['education'])
data['housing']=le.fit_transform(data['housing'])
data['loan']=le.fit_transform(data['loan'])
data['contact']=le.fit_transform(data['contact'])
data['month']=le.fit_transform(data['month'])
data['poutcome']=le.fit_transform(data['poutcome'])
data['default']=le.fit_transform(data['default'])

X = data.drop('y',axis=1)
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

model = LogisticRegression()
model.fit(X_train.to_dask_array(lengths=True), y_train.to_dask_array(lengths=True))

predictions = model.predict(X_test.to_dask_array(lengths=True)).compute()
print(predictions)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False Fa

In [48]:
from dask_ml.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_train_pred = model.predict(X_train.to_dask_array(lengths=True)).compute()
y_test_pred =  model.predict(X_test.to_dask_array(lengths=True)).compute()
  
# Test set performance
model_test_accuracy = accuracy_score(y_test.to_dask_array(lengths=True), y_test_pred) 
model_test_mse = mean_squared_error(y_test.to_dask_array(lengths=True), y_test_pred) 
model_test_mae  = mean_absolute_error(y_test.to_dask_array(lengths=True), y_test_pred) 
model_test_auc  = roc_auc_score(y_test.to_dask_array(lengths=True), y_test_pred) 

# Training set performance
model_train_accuracy = accuracy_score(y_train.to_dask_array(lengths=True), y_train_pred) 
model_train_mse = mean_squared_error(y_train.to_dask_array(lengths=True), y_train_pred) 
model_train_mae  = mean_absolute_error(y_train.to_dask_array(lengths=True), y_train_pred) 
model_train_auc  = roc_auc_score(y_train.to_dask_array(lengths=True), y_train_pred) 

print('Model performance for Training set')
print("- Accuracy: {:.4f}".format(model_train_accuracy))
print('- Mean Squared Error: {:4f}'.format(model_test_mse))
print('- Mean Absolute Error: {:4f}'.format(model_test_mae))
print('- AUC: {:4f}'.format(model_test_auc))

print('----------------------------------')

print('Model performance for Test set')
print('- Accuracy: {:.4f}'.format(model_test_accuracy) )
print('- Mean Squared Error: {:.4f}'.format(model_train_mse))
print('- Mean Absolute Error: {:.4f}'.format(model_train_mae))
print('- AUC: {:.4f}'.format(model_train_auc))

Model performance for Training set
- Accuracy: 0.8895
- Mean Squared Error: 0.110987
- Mean Absolute Error: 0.110987
- AUC: 0.592374
----------------------------------
Model performance for Test set
- Accuracy: 0.8890
- Mean Squared Error: 0.1105
- Mean Absolute Error: 0.1105
- AUC: 0.5857
